In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from sklearn.model_selection import train_test_split
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score,auc,roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
#n = 50 #number of records in file
#s = 10#desired sample size




In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int64',
        'RtpStateBitfield':                                     'float64',
        'IsSxsPassiveMode':                                     'int64',
        'DefaultBrowsersIdentifier':                            'float64',
        'AVProductStatesIdentifier':                            'float64',
        'AVProductsInstalled':                                  'float64',
        'AVProductsEnabled':                                    'float64',
        'HasTpm':                                               'int64',
        'CountryIdentifier':                                    'int64',
        'CityIdentifier':                                       'float64',
        'OrganizationIdentifier':                               'float64',
        'GeoNameIdentifier':                                    'float64',
        'LocaleEnglishNameIdentifier':                          'int64',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int64',
        'OsSuite':                                              'int64',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float64',
        'AutoSampleOptIn':                                      'int64',
        'PuaMode':                                              'category',
        'SMode':                                                'float64',
        'IeVerIdentifier':                                      'float64',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float64',
        'UacLuaenable':                                         'float64',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float64',
        'Census_OEMModelIdentifier':                            'float64',
        'Census_ProcessorCoreCount':                            'float64',
        'Census_ProcessorManufacturerIdentifier':               'float64',
        'Census_ProcessorModelIdentifier':                      'float64',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64',
        'Census_HasOpticalDiskDrive':                           'int64',
        'Census_TotalPhysicalRAM':                              'float64',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float64',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float64',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float64',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'BatteryNumberOfCharges':                               'float64',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int64',
        'Census_OSBuildRevision':                               'int64',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float64',
        'Census_OSUILocaleIdentifier':                          'int64',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int64',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float64',
        'Census_IsFlightsDisabled':                             'float64',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float64',
        'Census_FirmwareManufacturerIdentifier':                'float64',
        'Census_FirmwareVersionIdentifier':                     'float64',
        'Census_IsSecureBootEnabled':                           'int64',
        'Census_IsWIMBootEnabled':                              'float64',
        'Census_IsVirtualDevice':                               'float64',
        'Census_IsTouchEnabled':                                'int64',
        'Census_IsPenCapable':                                  'int64',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float64',
        'Wdft_IsGamer':                                         'float64',
        'Wdft_RegionIdentifier':                                'float64',
        'HasDetections':                                        'int64'
        }

In [ ]:
%%time
import dask.dataframe as dd
import gc

temp = pd.read_csv("../input/train.csv", dtype = dtypes,chunksize=250000)
for chunk in temp:
    #chunk=chunk.dropna()
    train_df=chunk
    break
gc.collect()

In [ ]:
train_df=train_df.drop(['ProductName','Census_IsWIMBootEnabled','Census_ThresholdOptIn','Census_IsFlightingInternal',
                 'Census_ProcessorClass','SmartScreen','PuaMode','Census_DeviceFamily',
                 'DefaultBrowsersIdentifier','MachineIdentifier',
                       'Census_GenuineStateName','Census_PowerPlatformRoleName',
                        'Census_FlightRing','OrganizationIdentifier','Census_ChassisTypeName',
                       'Census_ActivationChannel','Census_MDC2FormFactor','SkuEdition','SMode',
                        'Census_InternalBatteryType'],axis=1)
train_df=train_df.dropna()
train=pd.get_dummies(train_df)
del train_df


In [ ]:
train=train.dropna()
train1,validate1=train_test_split(train,test_size=0.3,random_state=100)
del train
gc.collect()

In [ ]:
'''Breaking into xtrain and ytrain for performing linear regression'''
train1=train1.dropna()
validate1=validate1.dropna()
x=train1.drop('HasDetections',axis=1)
y=train1.HasDetections
test_x=validate1.drop('HasDetections',axis=1)
test_y=validate1.HasDetections
del train1,validate1

In [ ]:
'''from sklearn.model_selection import GridSearchCV
params={'max_depth':list(range(1,12)),
        'criterion':['gini','entropy']}
base_estimator=DecisionTreeClassifier(random_state=100)
cv_model=GridSearchCV(base_estimator,param_grid=params,cv=10,scoring='roc_auc')
cv_model.fit(x,y)'''

In [ ]:
'''cv_model.best_params_'''

In [ ]:
fin_values=list()

In [ ]:
ml=list()

In [ ]:

test_temp=pd.read_csv("../input/test.csv",dtype=dtypes,chunksize=100000)
for chunk in test_temp:
    chunk=chunk.drop(['ProductName','OsVer','Census_IsWIMBootEnabled','Census_ThresholdOptIn','Census_IsFlightingInternal',
                 'Census_ProcessorClass','SmartScreen','PuaMode','Census_DeviceFamily','Census_PrimaryDiskTypeName',
                 'DefaultBrowsersIdentifier','Census_OSEdition','Census_OSSkuName','Processor',
                       'Census_GenuineStateName','Census_PowerPlatformRoleName',
                        'Census_FlightRing','Census_OSArchitecture','OrganizationIdentifier','Census_ChassisTypeName',
                       'Census_ActivationChannel','AvSigVersion','Census_MDC2FormFactor','SkuEdition','SMode',
                        'Census_InternalBatteryType','OsBuildLab','AppVersion','EngineVersion','Census_OSVersion','Census_OSBranch'],axis=1)
    ml.extend(chunk.MachineIdentifier)
    del chunk
  
 

In [ ]:
len(ml)

In [ ]:
test_temp=pd.read_csv("../input/test.csv",dtype=dtypes,chunksize=100000)
for chunk in test_temp:
    chunk=chunk.drop(['ProductName','Census_IsWIMBootEnabled','Census_ThresholdOptIn','Census_IsFlightingInternal',
                 'Census_ProcessorClass','SmartScreen','PuaMode','Census_DeviceFamily',
                 'DefaultBrowsersIdentifier','MachineIdentifier','OsVer','Processor','Census_PrimaryDiskTypeName',
                       'Census_GenuineStateName','Census_OSEdition','Census_OSSkuName','Census_PowerPlatformRoleName',
                        'Census_FlightRing','Census_OSArchitecture','OrganizationIdentifier','Census_ChassisTypeName',
                       'Census_ActivationChannel','AvSigVersion','Census_MDC2FormFactor','SkuEdition','SMode',
                        'Census_InternalBatteryType','OsBuildLab','AppVersion','EngineVersion','Census_OSVersion','Census_OSBranch'],axis=1)
    chunk=chunk.select_dtypes(exclude=('category')).fillna(chunk.select_dtypes(exclude=('category')).mean())
    chunk=pd.get_dummies(chunk)
    temp_cols = chunk.columns.tolist()
    del chunk
    break
  
 

In [ ]:
'''Fitting a classification model'''
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(random_state=100,max_depth=6,criterion='entropy')
model.fit(x[temp_cols],y)
pred=model.predict(test_x[temp_cols])
#final_acc=auc(test_y,pred)
#final_acc

In [ ]:
'''Visualising the AUC Area under the curve'''
pred_probs=pd.DataFrame(model.predict_proba(test_x[temp_cols]),columns=['Neg','Pos'])
fpr_knn,tpr_knn,thresholds=roc_curve(test_y,pred_probs['Pos'])
auc_knn=auc(fpr_knn,tpr_knn)
plt.plot(fpr_knn,tpr_knn)
plt.legend(['AUC %.2f' %auc_knn])
#del x,y,test_x,test_y
gc.collect()

In [ ]:
fin_values=list()

In [ ]:
test_temp=pd.read_csv("../input/test.csv",dtype=dtypes,chunksize=150000)
for chunk in test_temp:
    chunk=chunk.drop(['ProductName','OsVer','Census_IsWIMBootEnabled','Census_ThresholdOptIn','Census_IsFlightingInternal',
                 'Census_ProcessorClass','SmartScreen','PuaMode','Census_DeviceFamily',
                 'DefaultBrowsersIdentifier','Census_OSEdition','Census_OSSkuName','MachineIdentifier','Processor',
                       'Census_GenuineStateName','Census_PowerPlatformRoleName','Census_PrimaryDiskTypeName',
                        'Census_FlightRing','Census_OSArchitecture','OrganizationIdentifier','Census_ChassisTypeName',
                       'Census_ActivationChannel','AvSigVersion','Census_MDC2FormFactor','SkuEdition','SMode',
                        'Census_InternalBatteryType','OsBuildLab','AppVersion','EngineVersion','Census_OSVersion','Census_OSBranch'],axis=1)
    chunk=chunk.select_dtypes(exclude=('category')).fillna(chunk.select_dtypes(exclude=('category')).mean())
    chunk=pd.get_dummies(chunk)
    fin_values.extend(model.predict_proba(chunk[temp_cols])[:,1])
    del chunk

In [ ]:
len(fin_values)

In [ ]:
fin_pred=pd.DataFrame({'MachineIdentifier':ml,'HasDetections':fin_values})

In [ ]:
fin_pred.to_csv('FinalMalPred.csv',index=False)

In [ ]:
fin_pred.head(20)